scripts/rsl_rl/train.py
```python
@hydra_task_config(args_cli.task, "rsl_rl_cfg_entry_point")
def main(env_cfg: ManagerBasedRLEnvCfg | DirectRLEnvCfg | DirectMARLEnvCfg, agent_cfg: RslRlOnPolicyRunnerCfg):
    # ...
    # create isaac environment
    env = gym.make(args_cli.task, cfg=env_cfg, render_mode="rgb_array" if args_cli.video else None)
    # ...
    # wrap around environment for rsl-rl
    env = RslRlVecEnvWrapper(env)
    # create runner from rsl-rl
    runner = OnPolicyRunner(env, agent_cfg.to_dict(), log_dir=log_dir, device=agent_cfg.device)
    # ...
    # run training
    runner.learn(num_learning_iterations=agent_cfg.max_iterations, init_at_random_ep_len=True)
    # close the simulator
    env.close()
```

↓decorator 负责解析并提供 env_cfg 和 agent_cfg 

```python
def hydra_task_config(task_name: str, agent_cfg_entry_point: str) -> Callable:
    #...
    env_cfg, agent_cfg = register_task_to_hydra(task_name, agent_cfg_entry_point)
    #...
```

↓

```python
def register_task_to_hydra(
    task_name: str, agent_cfg_entry_point: str
) -> tuple[ManagerBasedRLEnvCfg | DirectRLEnvCfg, dict]：
    # ...
    env_cfg = load_cfg_from_registry(task_name, "env_cfg_entry_point")
    agent_cfg = load_cfg_from_registry(task_name, agent_cfg_entry_point)
    # ...
```

↓

```python
def load_cfg_from_registry(task_name: str, entry_point_key: str) -> dict | object:
    """It supports both YAML and Python configuration files.
    If the entry point is a YAML file, it is parsed into a dictionary.
    If the entry point is a Python class, it is instantiated and returned."""
    # obtain the configuration entry point
    cfg_entry_point = gym.spec(task_name).kwargs.get(entry_point_key)
    # 如果 cfg_entry_point 是一个以 .yaml 结尾的字符串
    # 如果 cfg_entry_point 是可调用的（例如是一个函数或类）
    # 如果 cfg_entry_point 是字符串（格式为 "module_name:attr_name"）
    # ...
```

.e.g
```python
import gymnasium as gym
from . import agents
from .zbot6b_env_v0 import ZbotBEnv, ZbotBEnvCfg
##
# Register Gym environments.
##
gym.register(
    id="Zbot-6b-walking-v0",
    entry_point="Zbot.tasks.moving.zbot6b_direct:ZbotBEnv",
    disable_env_checker=True,
    kwargs={
        "env_cfg_entry_point": ZbotBEnvCfg, 
        "rsl_rl_cfg_entry_point": f"{agents.__name__}.rsl_rl_ppo_cfg:ZbotSBFlatPPORunnerCfg",
    },
)
```